<a href="https://colab.research.google.com/github/andrewshandy/AvivaHackathon2021_text2sql/blob/main/seq2seq_text2sql.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
! nvidia-smi

Sun Apr 25 08:56:39 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8    27W / 149W |      3MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "1"

# Seq2seq

In [1]:
%matplotlib inline

In [2]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
import pandas as pd

In [4]:
torch.cuda.is_available()

True

In [6]:
df = pd.read_json('/content/train_spider.json')

In [7]:
df

,db_id,query,query_toks,query_toks_no_value,question,question_toks,sql
0,department_management,SELECT count(*) FROM head WHERE age > 56,"[SELECT, count, (, *, ), FROM, head, WHERE, ag...","[select, count, (, *, ), from, head, where, ag...",How many heads of the departments are older th...,"[How, many, heads, of, the, departments, are, ...","{'from': {'table_units': [['table_unit', 1]], ..."
1,department_management,"SELECT name , born_state , age FROM head ORD...","[SELECT, name, ,, born_state, ,, age, FROM, he...","[select, name, ,, born_state, ,, age, from, he...","List the name, born state and age of the heads...","[List, the, name, ,, born, state, and, age, of...","{'from': {'table_units': [['table_unit', 1]], ..."
2,department_management,"SELECT creation , name , budget_in_billions ...","[SELECT, creation, ,, name, ,, budget_in_billi...","[select, creation, ,, name, ,, budget_in_billi...","List the creation year, name and budget of eac...","[List, the, creation, year, ,, name, and, budg...","{'from': {'table_units': [['table_unit', 0]], ..."
3,department_management,"SELECT max(budget_in_billions) , min(budget_i...","[SELECT, max, (, budget_in_billions, ), ,, min...","[select, max, (, budget_in_billions, ), ,, min...",What are the maximum and minimum budget of the...,"[What, are, the, maximum, and, minimum, budget...","{'from': {'table_units': [['table_unit', 0]], ..."
4,department_management,SELECT avg(num_employees) FROM department WHER...,"[SELECT, avg, (, num_employees, ), FROM, depar...","[select, avg, (, num_employees, ), from, depar...",What is the average number of employees of the...,"[What, is, the, average, number, of, employees...","{'from': {'table_units': [['table_unit', 0]], ..."
...,...,...,...,...,...,...,...
6995,culture_company,SELECT T1.company_name FROM culture_company AS...,"[SELECT, T1.company_name, FROM, culture_compan...","[select, t1, ., company_name, from, culture_co...",What are all the company names that have a boo...,"[What, are, all, the, company, names, that, ha...","{'from': {'table_units': [['table_unit', 2], [..."
6996,culture_company,"SELECT T1.title , T3.book_title FROM movie AS...","[SELECT, T1.title, ,, T3.book_title, FROM, mov...","[select, t1, ., title, ,, t3, ., book_title, f...",Show the movie titles and book titles for all ...,"[Show, the, movie, titles, and, book, titles, ...","{'from': {'table_units': [['table_unit', 1], [..."
6997,culture_company,"SELECT T1.title , T3.book_title FROM movie AS...","[SELECT, T1.title, ,, T3.book_title, FROM, mov...","[select, t1, ., title, ,, t3, ., book_title, f...",What are the titles of movies and books corres...,"[What, are, the, titles, of, movies, and, book...","{'from': {'table_units': [['table_unit', 1], [..."
6998,culture_company,SELECT T2.company_name FROM movie AS T1 JOIN c...,"[SELECT, T2.company_name, FROM, movie, AS, T1,...","[select, t2, ., company_name, from, movie, as,...",Show all company names with a movie directed i...,"[Show, all, company, names, with, a, movie, di...","{'from': {'table_units': [['table_unit', 1], [..."


In [8]:
df_origin = pd.read_excel('/content/avia_distiled_sql.xlsx')
df_origin

,query,question
0,SELECT count(*) FROM flight WHERE velocity > ...,How many flights have a velocity larger than 200?
1,"SELECT vehicle_flight_number , date , pilot ...","List the vehicle flight number, date and pilot..."
2,"SELECT id , country , city , name FROM airp...","List the id, country, city and name of the air..."
3,SELECT max(group_equity_shareholding) FROM ope...,What is maximum group equity shareholding of t...
4,SELECT avg(velocity) FROM flight WHERE pilot ...,What is the velocity of the pilot named 'Thomp...
...,...,...
333,"SELECT T3.Pilot_name , T2.Fleet_Series FROM p...",Show the names of pilots and fleet series of t...
334,SELECT T2.Fleet_Series FROM pilot_record AS T1...,Show the fleet series of the aircrafts flied b...
335,"SELECT T2.Pilot_name , COUNT(*) FROM pilot_re...",Show the names of pilots and the number of rec...
336,"SELECT T2.Pilot_name , COUNT(*) FROM pilot_re...",Show names of pilots that have more than one r...


In [9]:
temp = pd.DataFrame((df_origin['query'] + '\t' + df_origin['question']))
temp

,0
0,SELECT count(*) FROM flight WHERE velocity > ...
1,"SELECT vehicle_flight_number , date , pilot ..."
2,"SELECT id , country , city , name FROM airp..."
3,SELECT max(group_equity_shareholding) FROM ope...
4,SELECT avg(velocity) FROM flight WHERE pilot ...
...,...
333,"SELECT T3.Pilot_name , T2.Fleet_Series FROM p..."
334,SELECT T2.Fleet_Series FROM pilot_record AS T1...
335,"SELECT T2.Pilot_name , COUNT(*) FROM pilot_re..."
336,"SELECT T2.Pilot_name , COUNT(*) FROM pilot_re..."


In [87]:
import numpy as np


In [12]:
new_df = pd.DataFrame((df['query'] + '\t' + df['question']))
d = new_df.append(temp,)

In [13]:
new_df = pd.DataFrame(d)
new_df.reset_index(None).drop(columns='index')

,0
0,SELECT count(*) FROM head WHERE age > 56\tHo...
1,"SELECT name , born_state , age FROM head ORD..."
2,"SELECT creation , name , budget_in_billions ..."
3,"SELECT max(budget_in_billions) , min(budget_i..."
4,SELECT avg(num_employees) FROM department WHER...
...,...
7333,"SELECT T3.Pilot_name , T2.Fleet_Series FROM p..."
7334,SELECT T2.Fleet_Series FROM pilot_record AS T1...
7335,"SELECT T2.Pilot_name , COUNT(*) FROM pilot_re..."
7336,"SELECT T2.Pilot_name , COUNT(*) FROM pilot_re..."


In [14]:
SOS_token = 0
EOS_token = 1


class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1
            
    def __repr__(self):
        most_popular_words = sorted(
            self.word2count.keys(), key=lambda word: self.word2count[word], reverse=True
        )[:10]
        most_popular_words = ", ".join(most_popular_words)
        return f"Language: {self.name} | Num words: {self.n_words} | Most popular: {most_popular_words}"

In [15]:
# "hello!" -> hello, ! 

def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

def normalizeString(s):
    s = unicodeToAscii(s)
    s = re.sub(r"([.!?])", r" \1", s)

    return s

Split into pairs


In [43]:
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")


    pairs = [[normalizeString(s) for s in l.split('\t')] for l in new_df[0]]


    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

In [44]:
MAX_LENGTH = 80

def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH 


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [45]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs


input_lang, output_lang, pairs = prepareData('SQL', 'eng', True)
print(random.choice(pairs))

Reading lines...
Read 7338 sentence pairs
Trimmed to 7322 sentence pairs
Counting words...
Counted words:
eng 4001
SQL 4870
['How many customers are there in the customer type with the most customers ?', 'SELECT count(*) FROM customers GROUP BY customer_type_code ORDER BY count(*) DESC LIMIT 1']


In [46]:
input_lang

Language: eng | Num words: 4001 | Most popular: the, of, ?, ., What, and, are, in, is, names

In [47]:
output_lang

Language: SQL | Num words: 4870 | Most popular: , T1, FROM, SELECT, T2, =, AS, WHERE, JOIN, ON

The Encoder
-----------

In [50]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super().__init__()
        self.hidden_size = hidden_size
        self.embedder = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embeddings = self.embedder(input).view(1, 1, -1)
        output, hidden = self.gru(embeddings, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

The Decoder
-----------





In [56]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super().__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(
                torch.cat((embedded[0], hidden[0]), 1)
            ), 
            dim=1
        )
        attn_applied = torch.bmm(
            attn_weights.unsqueeze(0),
            encoder_outputs.unsqueeze(0)
        )

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [57]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

Training the Model
------------------





In [58]:
teacher_forcing_ratio = 0.5


def train(
    input_tensor, 
    target_tensor,
    encoder, 
    decoder, 
    encoder_optimizer,
    decoder_optimizer, 
    criterion,
    max_length=MAX_LENGTH
):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)
    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

This is a helper function to print time elapsed and estimated time
remaining given the current time and progress %.




In [59]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

The whole training process looks like this:

-  Start a timer
-  Initialize optimizers and criterion
-  Create set of training pairs
-  Start empty losses array for plotting

Then we call ``train`` many times and occasionally print the progress (%
of examples, time so far, estimated time) and average loss.




In [60]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

Evaluation
=========




In [62]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

In [63]:
pairs[0]

['How many heads of the departments are older than 56  ?',
 'SELECT count(*) FROM head WHERE age  >  56']

We can evaluate random sentences from the training set and print out the
input, target, and output to make some subjective quality judgements:




In [64]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

Training and Evaluating
=======================




In [66]:
hidden_size = 256

encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.1).to(device)

Load pretrained model

In [ ]:
encoder1.load_state_dict(torch.load('/content/encoder_kaggle.pt')) 
attn_decoder1.load_state_dict(torch.load('/content/decoder_kaggle.pt'))

In [86]:
trainIters(encoder1, attn_decoder1, 20000, print_every=100)

In [ ]:
evaluateRandomly(encoder1, attn_decoder1)

In [68]:
ans = ''
for w in evaluate(encoder1, attn_decoder1, 'How many planes departure from Los Angeles ?')[0]:
    ans += (w) + ' '
ans

'SELECT count(*) FROM WHERE WHERE origin  =  "Los <EOS> '

In [84]:
for i in range (7000, 7015):
    print(pairs[i])

['how many airports are there in each country ?', 'SELECT count(*) ,  country FROM airport GROUP BY country']
['which countries have more than 2 airports ?', 'SELECT country FROM airport GROUP BY country HAVING count(*)  >  2']
['which pilot is in charge of the most number of flights ?', 'SELECT pilot FROM flight GROUP BY pilot ORDER BY count(*) DESC LIMIT 1']
['Find the name of the airport in the city of Goroka .', "SELECT name FROM airports WHERE city  =  'Goroka'"]
['What are the names of the airports in the city of Goroka ?', "SELECT name FROM airports WHERE city  =  'Goroka'"]
['Find the name, city, country, and altitude (or elevation) of the airports in the city of New York .', "SELECT name ,  city ,  country ,  elevation FROM airports WHERE city  =  'New York'"]
['What is the name, city, country, and elevation for every airport in the city of New York ?', "SELECT name ,  city ,  country ,  elevation FROM airports WHERE city  =  'New York'"]
['How many airlines are there ?', 'SEL

In [67]:
# encoder1.load_state_dict(torch.load('/content/encoder_kaggle.pt')) # можно и другую директорию, но вот это прямо внутри вашего гугл диска
# attn_decoder1.load_state_dict(torch.load('/content/decoder_kaggle.pt'))

<All keys matched successfully>

In [69]:
!pip install flask-ngrok

In [88]:
from flask_ngrok import run_with_ngrok
from flask import Flask
from flask import request, json, abort, jsonify
app = Flask(__name__)
run_with_ngrok(app)   #starts ngrok when the app is run
@app.route("/")
def home():
    ans = ''
    for w in evaluate(encoder1, attn_decoder1, 'How many planes departure from Los Angeles ?')[0]:
        ans += (w) + ' '
    return ans 
@app.route('/', methods=['POST'])
def parse_request():
    input_json = request.get_json(force=True) 
    print ('data from client:', input_json)
    text = input_json['text']
    ans = ''
    for word in evaluate(encoder1, attn_decoder1, text)[0]:
        ans += word + ' '

    print(ans)
    result = {"sql" : ans}
    return jsonify(result)
  
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://953a066ec0cd.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [25/Apr/2021 11:29:27] "POST / HTTP/1.1" 200 -


data from client: {'text': 'Which pilot is in charge of the most number of flights ?'}
SELECT pilot FROM pilot GROUP BY pilot ORDER BY count(*) DESC LIMIT 1 <EOS> 


...

data from client: {'text': 'Which pilot is in charge of the most number of flights ?'}
SELECT pilot FROM pilot GROUP BY ORDER BY count(*) DESC LIMIT 1 <EOS> 


127.0.0.1 - - [25/Apr/2021 11:40:48] "POST / HTTP/1.1" 200 -


data from client: {'text': 'Find the name of the airports located in Cuba or Argentina .'}
SELECT name FROM airports WHERE country  =  'Cuba' OR country  =  'Argentina' <EOS> 


127.0.0.1 - - [25/Apr/2021 11:41:05] "POST / HTTP/1.1" 200 -


data from client: {'text': 'How many airlines are there ?'}
SELECT count(*) FROM airlines <EOS> 
